In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset


In [2]:

dataset = load_dataset("hakurei/open-instruct-v1", split='train')
dataset.to_pandas().sample(20)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,instruction,output,input
448532,what exams do you recommend after azure fondam...,"To become an Azure developer, you may want to ...",
218960,What was the Cold War?,The Cold War was a time period where the US an...,
102881,Describe your favorite animal in one sentence.,My favorite animal is a cat.,
94708,Create an online poll on whether people prefer...,"<form action=""http://www.example.com/poll"" met...",
179703,What are some good techniques to bluff in poker.,Some good techniques to bluff in poker include...,
193304,Edit the following code snippet to make it cal...,"x = [1,2 ,3, 4, 5]\n\nimport statistics\n\nmed...",
254281,What are the Spice Girls nicknames?,"The Spice Girls iconic nicknames are Sporty, G...",
240283,"In C++, write a code snippet to print ""Welcome...","for (int i = 0; i < 10; i++) {\n cout << ""Wel...",
51816,Find all words in this sentence which end with...,"cat, chase, mouse",
37970,Record a rhyme with these two words.,"By the sea, the sun will shine and be free.","Sea, Sun"


In [3]:

def preprocess(example):
    example['prompt'] = f"{example['instruction']} {example['input']} {example['output']}"

    return example

def tokenize_datasets(dataset):
    tokenized_dataset = dataset.map(lambda example: tokenizer(example['prompt'], truncation=True, max_length=128), batched=True, remove_columns=['prompt'])

    return tokenized_dataset



In [4]:
dataset = dataset.map(preprocess, remove_columns=['instruction', 'input', 'output'])
dataset =  dataset.shuffle(42).select(range(100000)).train_test_split(test_size=0.1, seed=42)

In [5]:

train_dataset = dataset['train']
test_dataset = dataset['test']

In [6]:

MODEL_NAME = "microsoft/DialoGPT-medium"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

train_dataset = tokenize_datasets(train_dataset)
test_dataset = tokenize_datasets(test_dataset)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

traing_args = TrainingArguments(output_dir="models/diablo_gpt",
                                num_train_epochs=1,
                                per_device_train_batch_size=32,
                                per_device_eval_batch_size=32)\

trainer = Trainer(model=model,
                    args=traing_args,
                    train_dataset=train_dataset,
                    eval_dataset=test_dataset,
                    data_collator=data_collator)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [8]:
trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 39.06 MiB is free. Process 62333 has 14.71 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 10.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [9]:
# Get the trained checkpoint directly
model = AutoModelForCausalLM.from_pretrained("TheFuzzyScientist/diabloGPT_open-instruct")

config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [12]:

def generate_text(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=64, pad_token_id=tokenizer.eos_token_id)
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated[:generated.rfind('.')+1]

In [14]:

print(generate_text("What's the best way to cook chiken breast?"))

print(generate_text("Should I invest stocks?"))

print(generate_text("I need a place to go for this summer vacation, what locations would you recommend"))

print(generate_text("What's the fastest route from NY City to Boston?"))



What's the best way to cook chiken breast?  The best way to cook chiken breast is to season it with salt and pepper, then heat a pan over medium heat. Add a tablespoon of olive oil and cook for about 5 minutes, stirring occasionally.
Should I invest stocks?  Yes, it is a good idea to invest in stocks. It is important to understand the risks associated with investing in stocks and to make sure that you are taking the necessary precautions. It is also important to understand the potential returns and to make sure that you are making the right investment.
I need a place to go for this summer vacation, what locations would you recommend.  I would recommend visiting the beach in San Diego, California. It is a popular destination for vacationers and has a great view of the ocean.
What's the fastest route from NY City to Boston?  The fastest route from New York City to Boston is by taking the New York City subway. The subway takes about 3 hours and 15 minutes to get from the city center to th